In [2]:
!pip install "preliz[full,notebook]"

In [3]:
import preliz as pz
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import pymc as pm

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [5]:
url = "https://raw.githubusercontent.com/aahFancyGun/dataScienceClass/master/unit3/FL_Dashboard_AY2009_2010_Q1.csv"

data_loans = pd.read_csv(url)

data_loans

,2009-2010 Award Year FFEL Volume by School,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,Award Year Quarterly Activity (07/01/2009-09/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data Run: 4/5/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,FFEL SUBSIDIZED,NaN,NaN,NaN,NaN,...,FFEL PARENT PLUS,NaN,NaN,NaN,NaN,FFEL GRAD PLUS,NaN,NaN,NaN,NaN
4,OPE ID,School,State,Zip Code,School Type,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements,...,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3820,00393200,UNIVERSITY OF WYOMING,WY,820713663,PUBLIC,"2,699","2,735","$12,341,711.00","2,738","$6,819,888.00",...,195,207,"$1,830,536.00",209,"$950,682.00",58,65,"$740,877.00",66,"$416,236.00"
3821,00393300,WESTERN WYOMING COMMUNITY COLLEGE,WY,829010428,PUBLIC,214,214,"$731,883.00",215,"$359,742.00",...,2,2,"$13,300.00",2,"$6,650.00",-,-,$-,0,$-
3822,00728900,CENTRAL WYOMING COLLEGE,WY,825012215,PUBLIC,149,154,"$414,959.00",154,"$212,783.00",...,7,7,"$34,978.00",7,"$17,492.00",-,-,$-,0,$-
3823,00915700,WYOTECH,WY,820729519,PROPRIETARY,"1,049","1,099","$2,554,580.00","1,220","$1,498,483.00",...,478,490,"$7,567,391.00",522,"$4,033,026.00",-,-,$-,0,$-


In [15]:
print(data_loans.columns)

Index(['2009-2010 Award Year FFEL Volume by School', 'Unnamed: 1',
       'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24'],
      dtype='object')


In [12]:
studentLoanQuery = data_loans.query('Unnamed: 2 == "WY"')


NotImplementedError: 'AnnAssign' nodes are not implemented